<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/cifar10/CIFAR10_for_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Свёрточная нейронная сеть CIFAR-10**

In [17]:
import numpy
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras.optimizers import SGD

In [18]:
# Задаем seed для повторяемости результатов
numpy.random.seed(42)

In [19]:
# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 25
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

In [20]:
# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255
X_test = X_test / 255 

In [21]:
# Преобразуем метки в категории
Y_train = utils.to_categorical(y_train, nb_classes)
Y_test  = utils.to_categorical(y_test, nb_classes)

## Создание модели

In [22]:
# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой: 32 карты признаков, ядро свёртки 3x3, размер входных данных 32x32 в 3 канала
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='valid',
                        input_shape=(32, 32, 3), activation='relu',data_format="channels_last"))
# Второй сверточный слой
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='valid',data_format="channels_last"))
# Первый слой подвыборки: размер уменьшения слоя 2x2 с выбором максимального значения из этого квадрата
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last"))
# Слой регуляризации Dropout: при обучении случайным образом выключается 25% неронов - необходимо для предотвращения переобучения
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
# Четвертый сверточный слой
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu',data_format="channels_last"))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2),data_format="channels_last"))
# Слой регуляризации Dropout
model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

In [23]:

# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [24]:
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,        # перемешивание данных перед каждой эпохой обучения
              verbose=2)

Epoch 1/25
1407/1407 - 6s - loss: 1.7998 - accuracy: 0.3387 - val_loss: 1.3995 - val_accuracy: 0.4900
Epoch 2/25
1407/1407 - 6s - loss: 1.3713 - accuracy: 0.5039 - val_loss: 1.2020 - val_accuracy: 0.5598
Epoch 3/25
1407/1407 - 6s - loss: 1.2105 - accuracy: 0.5653 - val_loss: 1.0587 - val_accuracy: 0.6256
Epoch 4/25
1407/1407 - 6s - loss: 1.0931 - accuracy: 0.6138 - val_loss: 0.9794 - val_accuracy: 0.6624
Epoch 5/25
1407/1407 - 6s - loss: 1.0096 - accuracy: 0.6444 - val_loss: 0.9611 - val_accuracy: 0.6590
Epoch 6/25
1407/1407 - 6s - loss: 0.9552 - accuracy: 0.6627 - val_loss: 0.7893 - val_accuracy: 0.7262
Epoch 7/25
1407/1407 - 6s - loss: 0.9020 - accuracy: 0.6796 - val_loss: 0.8250 - val_accuracy: 0.7142
Epoch 8/25
1407/1407 - 6s - loss: 0.8700 - accuracy: 0.6951 - val_loss: 0.7456 - val_accuracy: 0.7422
Epoch 9/25
1407/1407 - 6s - loss: 0.8463 - accuracy: 0.7029 - val_loss: 0.8419 - val_accuracy: 0.7098
Epoch 10/25
1407/1407 - 6s - loss: 0.8253 - accuracy: 0.7109 - val_loss: 0.7639 - 

In [25]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

Точность работы на тестовых данных: 75.85%


In [26]:
print("Сохраняем сеть")
# Сохраняем сеть для последующего использования
# Генерируем описание модели в формате json
model_json = model.to_json()
json_file = open("cifar10_model.json", "w")
# Записываем архитектуру сети в файл
json_file.write(model_json)
json_file.close()
# Записываем данные о весах в файл
model.save_weights("cifar10_model.h5")
print("Сохранение сети завершено")

Сохраняем сеть
Сохранение сети завершено
